## 事前準備

In [1]:
 # google driveへの接続

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## CONST
定数の宣言  
ここは各自の環境に合わせて変更してください。ここではGoogleDriveを用いて行うことにします。  
ルートフォルダ内にdataフォルダを作成し、そこにダウンロードしたzipファイルを入れていください。

In [2]:
## rootフォルダ
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb"

## 出力フォルダ
OUTPUT_DIR = ROOT_DIR+ "/output"

## DATA
DATE_DIR = "/data"

In [ ]:
### フォルダ作成
import os
#### 上記のフォルダがなかった場合、作成する処理をここに入れる

## 処理

### import

In [3]:
### import
import shutil

import struct
from PIL import Image, ImageEnhance

import glob
import os

### make pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import pickle

### unzip

In [ ]:
etl9b_path = '/content/drive/MyDrive/code/ETLCDB/ETL9B.zip' # ここにetl9bのパスを入力

shutil.unpack_archive(etl9b_path ,OUTPUT_DIR+"/unzip")

### binary -> png

In [ ]:

dataset_dir = OUTPUT_DIR + '/unzip'

files = glob.glob(dataset_dir + '/ETL9B/*') # 解凍したバイナリファイルのpath

for fname in files:
    print('===== ===== =====')
    if fname == dataset_dir + "/ETL9B/ETL9BINFO": continue # 情報ファイルは飛ばす
    if fname == dataset_dir + "/ETL9B/output": continue
    print(fname)

    f = open(fname, 'rb')
    # f.seek(0) # ポインタを先頭に
    f.seek(576)
    while True:
        print('      ----- -----')
        s = f.read(576) 
        if not s: break
        # バイナリデータなのでPythonが理解できるように抽出
        r = struct.unpack('>2H4s504s64x', s)

        kanjicode_jis = r[1]
        #test
        # print(kanjicode_jis)
        # print(r)
        #/test

        # 画像を抽出
        iF = Image.frombytes('1', (64, 63), r[3], 'raw')
        # iF = Image.frombytes('1', (64, 63), r[3], 'bit')
        iP = iF.convert('L')
        dir = OUTPUT_DIR+ "/etl9b/" + str(kanjicode_jis)
        print(dir)
        if not os.path.exists(dir): 
          os.makedirs(dir)

        #####
        print("r[0]: " + str(r[0]))
        print("r[1]: " + str(r[1]))
        print("r[2]: " + str(r[2]))
        #####

        # fn = "{0:02x}-{1:02x}{2:04}.png".format(kanjicode_jis, r[0], r[2])
        # fn = "{0:02x}-{1:02x}.png".format(kanjicode_jis, r[0])
        fn = "{0:02x}-{1:02x}{2}.png".format(kanjicode_jis, r[0], r[2])
        print('fn: ' + fn)
        #####

        fullpath = dir + "/" + fn
        if os.path.exists(fullpath): continue
        enhancer = ImageEnhance.Brightness(iP)
        iE = enhancer.enhance(16)
        iE.save(fullpath, 'PNG')

### make-pickle

In [ ]:
# 保存ディレクトリと画像サイズの指定
out_dir = '/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb/output/etl9b'
im_size = 64 # 画像サイズ

save_file = "/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb/output/pickle/ETL9B.pickle" # 保存ファイル名と保存先
plt.figure(figsize=(9, 17)) 

# 画像ディレクトリから画像を読み込み開始
files = os.listdir(out_dir)
files_dir = [f for f in files if os.path.isdir(os.path.join(out_dir, f))]

result = []
for i, code in enumerate(files_dir):
    img_dir = out_dir + "/" + str(code)
    fs = glob.glob(img_dir + "/*")
    print("dir=",  img_dir)

    # 画像64X63を読み込んでグレイスケールに変換し64X64に整形
    for j, f in enumerate(fs):
      try:
        img = cv2.imread(f)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img_gray, (im_size, im_size))
        result.append([i, img])
      except :
        print("ERROR: "+ i)


# 「画像とラベル」のデータセットを保存
pickle.dump(result, open(save_file, "wb"))
plt.show()

#### make pickle(sub)
同時にdampするversion

In [ ]:
###test
dir = []

In [ ]:
# 保存ディレクトリと画像サイズの指定
out_dir = '/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb/output/etl9b'
im_size = 64 # 画像サイズ

save_file = "/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb/output/pickle/ETL9B.pickle" # 保存ファイル名と保存先
plt.figure(figsize=(9, 17)) 

# 画像ディレクトリから画像を読み込み開始
files = os.listdir(out_dir)
files_dir = [f for f in files if os.path.isdir(os.path.join(out_dir, f))]

result = []
for i, code in enumerate(files_dir):
    img_dir = out_dir + "/" + str(code)
    fs = glob.glob(img_dir + "/*")
    print("dir=",  img_dir)

    # 画像64X63を読み込んでグレイスケールに変換し64X64に整形
    for j, f in enumerate(fs):
      try:
        img = cv2.imread(f)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img_gray, (im_size, im_size))
        result.append([i, img])
        try:
          pickle.dump(result, open(save_file, "ab"))
        except:
          print("damp error:" + str(i))
      except :
        print("ERROR: "+ str(i))


# 「画像とラベル」のデータセットを保存
pickle.dump(result, open(save_file, "ab"))
plt.show()

### learning